DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Set up

In [11]:
# requirements
! pip install python-docx

DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
from docx import Document
from IPython.display import display, HTML

def read_word_document(file_path):
    document = Document(file_path)
    text_content = ""
    
    for paragraph in document.paragraphs:
        text_content += paragraph.text + "\n"

    return text_content



In [12]:
# Replace 'data/' with the actual path to your data folder
data_folder = '/home/alex/llm-applications/notebooks/data'

# List of Word document filenames
word_document_filenames = ['Raptor Contract.docx', 'Raptor Q&A2.docx', 'Robinson Advisory.docx', 'Robinson Q&A.docx']

# Read and display the content of each Word document
for filename in word_document_filenames:
    document_path = data_folder + filename
    
    # Read the Word document
    document_content = read_word_document(document_path)

    # Display the content in a separate Jupyter Notebook cell
    display(HTML(f"<h2>{filename}</h2><pre>{document_content}</pre>"))


PackageNotFoundError: Package not found at '/home/alex/llm-applications/notebooks/dataRaptor Contract.docx'

#### Sections

Now that we have a dataset of all the paths to the html files, we're going to develop some functions that can appropriately extract the content from these files. We want to do this in a generalized manner so that we can perform this extraction across all of our docs pages (and so you can use it for your own data sources). Our process is to first identify the sections in our html page and then extract the text in between them. We save all of this into a list of dictionaries that map the text within a section to a specific url with a section anchor id.

In [9]:
import matplotlib.pyplot as plt
from rag.data import extract_sections

ModuleNotFoundError: No module named 'rag.data'